<a href="https://colab.research.google.com/github/vishwaraj06/Python_VN2/blob/main/Copy_of_raj_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!apt-get install openjdk-8-jdk-headless -qq > /dev/null


!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz



!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()

findspark.find()

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Mounted at /content/drive


In [ ]:
df=spark.read.csv("1000 BT Records.csv",inferSchema=True,header=True)

In [ ]:
df.select("Date").show(5)

+-----------+
|       Date|
+-----------+
|20-Aug-2020|
|20-Aug-2020|
|20-Aug-2020|
|21-Aug-2020|
|21-Aug-2020|
+-----------+
only showing top 5 rows



In [ ]:
df2=df.select("Date","Description","Balance")

In [ ]:
df2.show(5)

+-----------+-----------+----------+
|       Date|Description|   Balance|
+-----------+-----------+----------+
|20-Aug-2020|     Cheque| 83,839.30|
|20-Aug-2020|        ATM|231,683.50|
|20-Aug-2020|       IMPS|248,875.50|
|21-Aug-2020|   Transfer|248,883.50|
|21-Aug-2020|   Purchase|165,922.33|
+-----------+-----------+----------+
only showing top 5 rows



In [ ]:
from datetime import datetime 
from pyspark.sql.functions import col, udf 
from pyspark.sql.types import *
from pyspark.sql.types import IntegerType,BooleanType,DateType

In [ ]:
df2.filter(col("Withdrawls").isNotNull()).count()

1000

In [ ]:
from datetime import datetime

def try_parsing_date(text):
    for fmt in ('%Y-%m-%d', '%d.%m.%Y', '%d/%m/%Y','%dd-%mm-%YYYY','%d-%m-%Y','%d-%b-%Y'):
        try:
          new=datetime.strptime(text, fmt)
          new1=new.strftime('%d-%m-%Y')
          return new1
        except ValueError:
            pass
    raise ValueError('no valid date format found')

In [ ]:
date_udf=udf(lambda x:try_parsing_date(x))

In [ ]:
date_cha = df.select(col('date'),date_udf(col('date').alias('date_newwwww')))

In [ ]:
date_cha.show(5)

+-----------+------------------------------+
|       date|<lambda>(date AS date_newwwww)|
+-----------+------------------------------+
|20-Aug-2020|                    20-08-2020|
|20-Aug-2020|                    20-08-2020|
|20-Aug-2020|                    20-08-2020|
|21-Aug-2020|                    21-08-2020|
|21-Aug-2020|                    21-08-2020|
+-----------+------------------------------+
only showing top 5 rows



In [ ]:
pip install validate_email

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validate-email: filename=validate_email-1.3-py3-none-any.whl size=5482 sha256=f6ed2fdcd1258b59690c0faa971697429a2c8a23f3465c5d6cf27a4b2e4b533d
  Stored in directory: /root/.cache/pip/wheels/ff/8f/92/c43287715852eaa75e0d8aa1941c481072b4a82c4f4975074e
Successfully built validate-email


In [ ]:
from validate_email import validate_email
is_valid = validate_email("valid@mail.com")

In [ ]:
new_mail=spark.read.csv("data_prep.csv",inferSchema=True,header=True)

In [ ]:
new_mail.show(5)

+------+---------------+----------+------+--------+
|userid|        emailid|      date|amount|discount|
+------+---------------+----------+------+--------+
|     1|test1@gmail.com|01-01-2019|   100|     1.0|
|     2|test2@gmail.com|02-01-2019|   100|     1.0|
|     3|test3@gmail.com|03-01-2019|   101|     2.0|
|     4|test4@gmail.com|04-01-2019|   102|    10.0|
|     5|test5@gmail.com|05-01-2019| 102.5|     2.0|
+------+---------------+----------+------+--------+
only showing top 5 rows



In [ ]:
def email_validation(str):
  is_valid = validate_email(str)
  if is_valid:
    return str
  else:
    "error"

In [ ]:
email_udf = udf(lambda x : email_validation(x))

In [ ]:
After_Validation = new_mail.select(col('emailid'),email_udf(col('emailid').alias('emailid_validated')))

In [ ]:
After_Validation.show(5)

+---------------+--------------------------------------+
|        emailid|<lambda>(emailid AS emailid_validated)|
+---------------+--------------------------------------+
|test1@gmail.com|                       test1@gmail.com|
|test2@gmail.com|                       test2@gmail.com|
|test3@gmail.com|                       test3@gmail.com|
|test4@gmail.com|                       test4@gmail.com|
|test5@gmail.com|                       test5@gmail.com|
+---------------+--------------------------------------+
only showing top 5 rows



In [ ]:
def zero_fun(x):
  try:
    int_x = int(x)
    print(int_x)
    if int_x ==0:
      return 'ERROR'
    return x
  except ValueError:
    if x =='00.00':
      return 'NIL'
    return x

In [ ]:
zero_UDF = udf(lambda x : zero_fun(x))

In [ ]:
Zero_udf=df.select(df.Date, zero_UDF(col("Balance")).alias("Bal"))

In [ ]:
Zero_udf.show(13)

+-----------+------------+
|       Date|         Bal|
+-----------+------------+
|20-Aug-2020|   83,839.30|
|20-Aug-2020|  231,683.50|
|20-Aug-2020|  248,875.50|
|21-Aug-2020|  248,883.50|
|21-Aug-2020|  165,922.33|
|21-Aug-2020|  190,381.73|
|21-Aug-2020|  171,343.56|
|21-Aug-2020|  477,221.56|
|21-Aug-2020|1,621,559.36|
|21-Aug-2020|1,216,169.52|
|21-Aug-2020|1,216,202.60|
|21-Aug-2020|1,236,216.62|
|21-Aug-2020|         NIL|
+-----------+------------+
only showing top 13 rows



In [ ]:
parquet_df=spark.read.parquet("userdata4.parquet")


In [ ]:
parquet_df.show()

+-------------------+---+----------+----------+--------------------+------+---------------+-------------------+-------------+----------+---------+--------------------+--------+
|  registration_dttm| id|first_name| last_name|               email|gender|     ip_address|                 cc|      country| birthdate|   salary|               title|comments|
+-------------------+---+----------+----------+--------------------+------+---------------+-------------------+-------------+----------+---------+--------------------+--------+
|2016-02-04 15:28:05|  1|    Howard|    Morgan|hmorgan0@typepad.com|      | 158.178.195.62|                   |     Colombia| 12/2/1992|     null|     Data Coordiator|        |
|2016-02-04 10:28:40|  2|   Jessica|   Schmidt|jschmidt1@google.com|Female| 168.118.247.35|   3565285464047941|   Luxembourg| 4/14/1995|222396.46|      Research Nurse|     nil|
|2016-02-04 18:01:36|  3|   Beverly|    Flores|bflores2@wikipedi...|Female|   51.97.88.173|                   |    

In [ ]:
json_df=spark.read.json("/content/part-00000-74edbdc7-7bd4-43c1-aabd-2d069628baf8-c000.json")

In [ ]:
json_df.show(5)

+------+----------+--------+---------------+-----+------+----+
|amount|      date|discount|        emailid|month|userid|year|
+------+----------+--------+---------------+-----+------+----+
|   100|01-01-2019|     1.0|test1@gmail.com|   01|     1|2019|
|   100|02-01-2019|     1.0|test2@gmail.com|   01|     2|2019|
|   101|03-01-2019|     2.0|test3@gmail.com|   01|     3|2019|
|   102|04-01-2019|    10.0|test4@gmail.com|   01|     4|2019|
| 102.5|05-01-2019|     2.0|test5@gmail.com|   01|     5|2019|
+------+----------+--------+---------------+-----+------+----+
only showing top 5 rows



In [ ]:
json_df.write.parquet("Json_converted_Parquet")

In [ ]:
new_par=spark.read.parquet("/content/Json_converted_Parquet/part-00000-323b0fae-f621-4333-8b8d-19f684b957ae-c000.snappy.parquet").show(5)

+------+----------+--------+---------------+-----+------+----+
|amount|      date|discount|        emailid|month|userid|year|
+------+----------+--------+---------------+-----+------+----+
|   100|01-01-2019|     1.0|test1@gmail.com|   01|     1|2019|
|   100|02-01-2019|     1.0|test2@gmail.com|   01|     2|2019|
|   101|03-01-2019|     2.0|test3@gmail.com|   01|     3|2019|
|   102|04-01-2019|    10.0|test4@gmail.com|   01|     4|2019|
| 102.5|05-01-2019|     2.0|test5@gmail.com|   01|     5|2019|
+------+----------+--------+---------------+-----+------+----+
only showing top 5 rows



In [ ]:
parquet_df.write.json("parquet_converted_json")

In [ ]:
new_json=spark.read.json("/content/parquet_converted_json/part-00000-da4e0361-0cd8-4767-8fb2-32c0d2279d11-c000.json")

In [ ]:
new_json.show(5)

+---------+-------------------+--------+----------+--------------------+----------+------+---+---------------+---------+--------------------+---------+--------------------+
|birthdate|                 cc|comments|   country|               email|first_name|gender| id|     ip_address|last_name|   registration_dttm|   salary|               title|
+---------+-------------------+--------+----------+--------------------+----------+------+---+---------------+---------+--------------------+---------+--------------------+
|12/2/1992|                   |        |  Colombia|hmorgan0@typepad.com|    Howard|      |  1| 158.178.195.62|   Morgan|2016-02-04T15:28:...|     null|     Data Coordiator|
|4/14/1995|   3565285464047941|     nil|Luxembourg|jschmidt1@google.com|   Jessica|Female|  2| 168.118.247.35|  Schmidt|2016-02-04T10:28:...|222396.46|      Research Nurse|
|2/15/1965|                   |        |    Sweden|bflores2@wikipedi...|   Beverly|Female|  3|   51.97.88.173|   Flores|2016-02-04T18:0

In [ ]:
df.write.json("CSV_converted_json")

AnalysisException: ignored

In [ ]:
new_update_json=spark.read.json("/content/CSV_converted_json/part-00000-7f9cedb3-8a90-473a-9963-89c9fdcc0f43-c000.json")

In [ ]:
new_update_json.show(5)

+----------+-----------+----------+-----------+----------+
|   Balance|       Date|  Deposits|Description|Withdrawls|
+----------+-----------+----------+-----------+----------+
| 83,839.30|20-Aug-2020|  3,391.02|     Cheque|     00.00|
|231,683.50|20-Aug-2020|147,844.20|        ATM|     00.00|
|248,875.50|20-Aug-2020| 17,192.00|       IMPS|     00.00|
|248,883.50|21-Aug-2020|     08.00|   Transfer|     00.00|
|165,922.33|21-Aug-2020|     00.00|   Purchase| 82,961.17|
+----------+-----------+----------+-----------+----------+
only showing top 5 rows

